# 4 
## 4.1 **准自然实验设计：干旱事件 + 双重差分（DiD）**

本研究以2010年干旱作为处理组识别事件，将县份分为“严重干旱组”（SPEI≤–1）与“轻微干旱组”（SPEI > –1）进行对比分析。采用固定效应面板回归模型

In [ ]:
# 导入相关库
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

In [6]:
df = pd.read_csv("data/guizhou_panel_cleaned.csv")

### 4.1.1 构造县ID、处理组、Post、交互项变量

In [7]:
# 1.1 县名转为唯一编码
df['county_id'] = df['ENG_NAME'].astype('category').cat.codes

# 1.2 构造 treatment dummy（四分位中最高等级为受灾县）
df['drought_treated'] = (df['drought_class'] == 4).astype(int)

# 1.3 Post变量 + 事件期
df['post_2010'] = (df['year'] >= 2010).astype(int)
df['event_time'] = df['year'] - 2010

# 1.4 基础交互项
df['drought_post'] = df['drought_treated'] * df['post_2010']

### 4.1.2 构造空间异质性变量

In [8]:
# 高农业依赖（ cropland_pct 前25%）
threshold = df['cropland_pct'].quantile(0.75)
df['high_agriculture'] = (df['cropland_pct'] >= threshold).astype(int)

# 交互项
df['drought_post_high_agri'] = df['drought_post'] * df['high_agriculture']